In [1]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import time
import math
import community as community_louvain
import networkx.algorithms.community as modularity
from networkx.algorithms.community.quality import modularity
from collections import defaultdict


G = nx.read_edgelist("C:/Users/gnssl/Downloads/facebook_combined.txt", 
                     create_using=nx.Graph(), nodetype=int)

partition_louvain = community_louvain.best_partition(G)
len_partition = len(partition_louvain)

population_size = 200
generations = 500
crossover_prob = 1.0
mutation_prob = 0.01
local_optimization_frequency=5

min_partition_size = 5
max_partition_size = 15

def initialize_population():
    population = []
    for idx in range(population_size):
        
        partition = [random.randint(0, max_partition_size) for _ in range(len(G.nodes))]
        population.append(partition)
    return population

def compute_fitness(partition):
    communities = {key: value for key, value in zip(partition_louvain.keys(), partition)}
    new_dict = defaultdict(list)

    for key, value in communities.items():
        new_dict[value].append(key)

    result_dict = dict(new_dict)
    fitness = nx.community.modularity(G, result_dict.values())
    return fitness


def select_parents(population, fitness_values):
    parent1, parent2 = random.sample(population, 2)
    if compute_fitness(parent1) > compute_fitness(parent2):
        return parent1
    else:
        return parent2

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(individual, mutation_prob, max_partition_size):
    for i in range(len(individual)):
        if random.random() < mutation_prob:
            # Mutate the community assignment of the node to a random community
            individual[i] = random.randint(0, max_partition_size - 1)
    return individual

def partition_to_communities(partition):
    communities = {}
    for node, community_id in enumerate(partition):
        if community_id in communities:
            communities[community_id].add(node)
        else:
            communities[community_id] = {node}
    return list(communities.values())



def genetic_algorithm():
    population = initialize_population()
    max_fitness_values = []  # Store max fitness values for each generation

    for generation in range(generations):
        fitness_values = [compute_fitness(partition) for partition in population]
        max_fitness = max(fitness_values)
        max_fitness_values.append(max_fitness)
        avg_fitness = sum(fitness_values) / len(fitness_values)
        print('Generation', generation , 'Max:', max_fitness)
        print('Generation', generation , 'Avg:', avg_fitness)

        new_population = []
        for _ in range(population_size // 2):
            parent1 = select_parents(population, fitness_values)
            parent2 = select_parents(population, fitness_values)
            if random.random() < crossover_prob:
                child1, child2 = crossover(parent1, parent2)
                
                child1 = mutate(child1, mutation_prob, max_partition_size)
                child2 = mutate(child2, mutation_prob, max_partition_size)
                
                min_fitness_index = min(range(len(fitness_values)), key=fitness_values.__getitem__)
                population[min_fitness_index] = child1
                second_min_fitness_index = min(range(len(fitness_values)), key=lambda x: fitness_values[x] if x != min_fitness_index else float('inf'))
                population[second_min_fitness_index] = child2
                
        
        if generation % local_optimization_frequency == 0:
            random_idx = random.randint(1,population_size//2)
            for idx in range(0, random_idx, population_size//random_idx):
                partition_louvain = community_louvain.best_partition(G)
                partition = [partition_louvain[node] for node in G.nodes]
                population[idx] = partition
       
    # Select the best partition from the final population
    best_partition = max(population, key=compute_fitness)
    return best_partition, max_fitness_values

start = time.time()
best_partition, max_fitness_values = genetic_algorithm()

best_communities = partition_to_communities(best_partition)
end = time.time()
print('time:', end - start)

print("Best Partition:", best_communities)

Generation 1 Max: 0.0012990037707200633
Generation 1 Avg: -0.0005783147818080893
Generation 2 Max: 0.8349652587713134
Generation 2 Avg: 0.0036201532401018578
Generation 3 Max: 0.8349652587713134
Generation 3 Avg: 0.003649922655307189
Generation 4 Max: 0.8349652587713134
Generation 4 Avg: 0.0036725506986493707
Generation 5 Max: 0.8349652587713134
Generation 5 Avg: 0.003689058895639598
Generation 6 Max: 0.8349652587713134
Generation 6 Avg: 0.0037089828109312718
Generation 7 Max: 0.8349699474521806
Generation 7 Avg: 0.08724972449322785
Generation 8 Max: 0.8349699474521806
Generation 8 Avg: 0.08726256610131893
Generation 9 Max: 0.8349699474521806
Generation 9 Avg: 0.08727741910244377
Generation 10 Max: 0.8349699474521806
Generation 10 Avg: 0.09126390825906483
Generation 11 Max: 0.8349699474521806
Generation 11 Avg: 0.0912776288177205
Generation 12 Max: 0.8349699474521806
Generation 12 Avg: 0.09902229371107808
Generation 13 Max: 0.8349699474521806
Generation 13 Avg: 0.09904075543935703
Gene

Generation 109 Max: 0.8358143525765493
Generation 109 Avg: 0.7148339829984018
Generation 110 Max: 0.8358143525765493
Generation 110 Avg: 0.7198958321002845
Generation 111 Max: 0.8358143525765493
Generation 111 Avg: 0.7246157928091556
Generation 112 Max: 0.8358131984693624
Generation 112 Avg: 0.7285835134431173
Generation 113 Max: 0.8358131984693624
Generation 113 Avg: 0.7327449908162461
Generation 114 Max: 0.8358131984693624
Generation 114 Avg: 0.7370336498479639
Generation 115 Max: 0.8358131984693624
Generation 115 Avg: 0.7413714860969131
Generation 116 Max: 0.8358131984693624
Generation 116 Avg: 0.7444458310293731
Generation 117 Max: 0.8358131984693624
Generation 117 Avg: 0.7482279039872908
Generation 118 Max: 0.8358131984693624
Generation 118 Avg: 0.7520534528449988
Generation 119 Max: 0.8358131984693624
Generation 119 Avg: 0.7558705870147244
Generation 120 Max: 0.8358131984693624
Generation 120 Avg: 0.759596335845182
Generation 121 Max: 0.8358131984693624
Generation 121 Avg: 0.7628

Generation 215 Max: 0.8357711022652161
Generation 215 Avg: 0.81048899010028
Generation 216 Max: 0.8357711022652161
Generation 216 Avg: 0.8104016793492801
Generation 217 Max: 0.83566964423358
Generation 217 Avg: 0.8114304192140436
Generation 218 Max: 0.83566964423358
Generation 218 Avg: 0.8114654164673121
Generation 219 Max: 0.83566964423358
Generation 219 Avg: 0.8107253081622767
Generation 220 Max: 0.83566964423358
Generation 220 Avg: 0.8112034663930864
Generation 221 Max: 0.83566964423358
Generation 221 Avg: 0.8118278133209508
Generation 222 Max: 0.83566964423358
Generation 222 Avg: 0.8119077010471969
Generation 223 Max: 0.83566964423358
Generation 223 Avg: 0.8122823707823096
Generation 224 Max: 0.83566964423358
Generation 224 Avg: 0.8126558606759573
Generation 225 Max: 0.83566964423358
Generation 225 Avg: 0.8121148200811196
Generation 226 Max: 0.83566964423358
Generation 226 Avg: 0.8118145989021995
Generation 227 Max: 0.8356849228273044
Generation 227 Avg: 0.8119095219386872
Generati

Generation 321 Max: 0.8357285843655891
Generation 321 Avg: 0.8184923530664422
Generation 322 Max: 0.8357285843655891
Generation 322 Avg: 0.8189306595753298
Generation 323 Max: 0.8357285843655891
Generation 323 Avg: 0.8189436258777341
Generation 324 Max: 0.8357285843655891
Generation 324 Avg: 0.8184079143664093
Generation 325 Max: 0.8357285843655891
Generation 325 Avg: 0.8186785551484118
Generation 326 Max: 0.8357285843655891
Generation 326 Avg: 0.8177608304759354
Generation 327 Max: 0.8357285843655891
Generation 327 Avg: 0.8188885033617843
Generation 328 Max: 0.8357285843655891
Generation 328 Avg: 0.8184829115387248
Generation 329 Max: 0.8357285843655891
Generation 329 Avg: 0.8190310949954249
Generation 330 Max: 0.8357285843655891
Generation 330 Avg: 0.8182737150100705
Generation 331 Max: 0.8357285843655891
Generation 331 Avg: 0.8192180200166876
Generation 332 Max: 0.8350077731386147
Generation 332 Avg: 0.8190600262821921
Generation 333 Max: 0.8350077731386147
Generation 333 Avg: 0.819

Generation 427 Max: 0.8358107564119515
Generation 427 Avg: 0.8208351719117146
Generation 428 Max: 0.8358107564119515
Generation 428 Avg: 0.8219517643859403
Generation 429 Max: 0.8358107564119515
Generation 429 Avg: 0.822090866698527
Generation 430 Max: 0.8358107564119515
Generation 430 Avg: 0.8217867731274501
Generation 431 Max: 0.8358107564119515
Generation 431 Avg: 0.8218274695848652
Generation 432 Max: 0.8358107564119515
Generation 432 Avg: 0.822067305233266
Generation 433 Max: 0.8358107564119515
Generation 433 Avg: 0.8221181488704883
Generation 434 Max: 0.8358107564119515
Generation 434 Avg: 0.8220109951798764
Generation 435 Max: 0.8358107564119515
Generation 435 Avg: 0.8221498659176771
Generation 436 Max: 0.8358107564119515
Generation 436 Avg: 0.8214051218081234
Generation 437 Max: 0.8358107564119515
Generation 437 Avg: 0.8221338850368847
Generation 438 Max: 0.8358107564119515
Generation 438 Avg: 0.8208916733506924
Generation 439 Max: 0.8358107564119515
Generation 439 Avg: 0.82094